# Simulação de Redes Booleanas

O presente *notebook* tem como objetivo explorar a simulação síncrona de redes booleanas, que são modeladas com base em sistemas biológicos e frequentemente servem de base para experimentos *in silico* que visam entender a dinâmica da interação entre genes, moléculas ou outros componentes que façam parte de sistemas biológicos. 

Sabendo que simular $N$ possíveis estados é uma tarefa de complexidade exponencial, visto que temos $2^{N}$ possíveis estados, faz-se necessário utilizar estratégias diminuam tempo de execução de execução. Como o principal objetivo das simulações é obter um conjunto de probabilidades associada a cada agente biológico estudado e a exploração dos estados da rede é independente de outros estados que não o anterior, este problema se torna candidato a otimização utilizando computação paralela.

# Dependencies

Plugins necessários para execução de código em CUDA com a *cell magic* `%%gpu`

In [ ]:
!pip install git+git://github.com/canesche/nvcc4jupyter.git
!git clone https://github.com/canesche/nvcc4jupyter
!git clone https://github.com/lucas-t-reis/research.git
%load_ext nvcc_plugin

  Cloning git://github.com/canesche/nvcc4jupyter.git to /tmp/pip-req-build-23t_mii0
  Running command git clone -q git://github.com/canesche/nvcc4jupyter.git /tmp/pip-req-build-23t_mii0
  Created wheel for ColabPlugin: filename=ColabPlugin-blind-cp37-none-any.whl size=12717 sha256=edfc012e82d34954b2d0320c8b8d321b76175442859d8c8a4d128e5aab73251e
  Stored in directory: /tmp/pip-ephem-wheel-cache-05wfv2gu/wheels/87/ae/09/21c6e192181a0472e20ddd1d5212e2cbb113f00ebe14330d0d
Successfully built ColabPlugin
Cloning into 'nvcc4jupyter'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 1147 (delta 100), reused 328 (delta 74), pack-reused 785
Receiving objects: 100% (1147/1147), 35.71 MiB | 19.57 MiB/s, done.
Resolving deltas: 100% (554/554), done.
Cloning into 'research'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100

GPU utilizada nos experimentos

In [ ]:
!nvidia-smi

Thu Mar 25 21:43:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Neste arquivo de cabeçalho, se encontram:

- Macro de verificação de erros para GPU;
- Encapsulamento da implementação de um Cronômetro para CPU e para GPU;
- Função auxiliar para imprimir os tempos de cada execução em um arquivo `.csv`;
- Função auxiliar para imprimir o histograma obtido após as simulações;

In [ ]:
%%writefile profiler.h
#ifndef PROFILER_H
#define PROFILER_H

#include<chrono>
#include<fstream>

// Careful for overflow above 9. Would require changing int to long long
#define MAX_POWER10 9

//Macro for checking cuda errors following a cuda launch or api call
#define cudaCheckError() {                                                      \
 cudaError_t e=cudaGetLastError();                                              \
 if(e!=cudaSuccess) {                                                           \
   printf("Cuda failure %s:%d: '%s'\n",__FILE__,__LINE__,cudaGetErrorString(e));\
   exit(0);                                                                     \
 }                                                                              \
}

class CPU_Stopwatch {
    
    typedef std::chrono::steady_clock clock;
    
    private:
      std::chrono::time_point<clock> start;

    public:
      void reset() { start = clock::now(); }
      CPU_Stopwatch() { reset(); }
      auto stop() { 
          std::chrono::duration<float> elapsed_time = clock::now() - start;
          return elapsed_time.count() * 1000;
      }
};

class GPU_Stopwatch {
    
    private:
      cudaEvent_t start;
      cudaEvent_t end;

    public:
      GPU_Stopwatch() {
          
          cudaEventCreate(&start);
          cudaEventCreate(&end);
          
          cudaEventRecord(start);
      }
      float stop() {

          cudaEventRecord(end);

          cudaEventSynchronize(end);
          float milliseconds = -1;
          cudaEventElapsedTime(&milliseconds, start, end);

          return milliseconds;

      }
    
};

void write_csv(const char * fileName, float profiling_times[], size_t n) {
    
    std::ofstream csv(fileName, std::ofstream::app);
    for(int i=0; i<n; i++) {
      if(i==n-1)
        csv << profiling_times[i] << "\n";
      else
        csv << profiling_times[i] << ",";    
    }    
}
void reduce_histogram(int *histogram, int *thread_histograms, size_t size, size_t n_genes) {

    //#pragma unroll
    for(int i=0; i<size; i++)
        histogram[i%n_genes] += thread_histograms[i];
}
void print_histogram(int *histogram, size_t n) {
    printf("Histogram [");
    for(int i=0; i<n; i++) {
        if(i==n-1) printf("%.2lf]\n\n", histogram[i]/(1000000000.0));
        else printf("%.2lf,", histogram[i]/(1000000000.0));
    }
}

#endif

Writing profiler.h


# Node activation estimation based on Simulation

## Equation parser

A simple python implementation to convert `.bnet` files into `c++` headers with a given boolean network configuration.

In [ ]:
%%writefile equation_to_code.py

import random
import sys
import re

# Removes non-alphanumeric characters from the String.
def filter(t):
    if t[0].isnumeric():
        print(  'Gene names can\'t start with numbers.' 
                'Try renaming the gene %s on all of its occurrences'
                '\nAborting' % t)
        exit()
    
    t = re.sub('[^a-zA-Z0-9\-\_]', '', t)
    return t.replace('-', '_')

#file = 'research/boolean-networks/networks/40-100/Apoptosis network/Apoptosis network.txt'
file = sys.argv[1]
skip = ['\n', '#']
operators = {'AND':' && ', 'NOT':'!', 'OR':' || ', '(':'(', ')':')'}
fixed_value = ['TRUE', 'FALSE', 'RANDOM']

node_idx = {}
fixed_node = {}

i = 0
# Assigning index to nodes
with open(file) as input:
    for line in input.readlines():
        if line[0] in skip:
            continue

        node, equation = line.split('=')
        
        # Reading fixed states
        if equation.strip() in fixed_value:
            fixed_node[filter(node)] = equation.strip()
            print('Fixed node %s with %s value' % (filter(node), fixed_node[filter(node)]))
            continue

        node_idx[filter(node)] = i
        i += 1

# Original nodes are needed for registers due to undeclared nodes within equations
original_nodes = node_idx.copy()

N = i
code = ''
with open(file) as input:
    for line in input.readlines():
        if line[0] in skip:
            continue

        node, equation = line.split('=')
        
        if filter(node) in fixed_node:
            continue
        
        index = str( node_idx[filter(node)] )
        code += '\t\ttemp_state[offset+'+index+'] = '
        
        for token in equation.split():
            
            # Fixed Nodes
            if filter(token) in fixed_node:
                value = fixed_node[filter(token)]
                if value == 'RANDOM':
                    value = 'TRUE' if random.randint(0,1) else 'FALSE'
                    fixed_node[filter(token)] = value
                    print('Setting RANDOM valued node %s to %s' % (token, fixed_node[filter(token)]))
                
                code += value.lower()

            # Nodes
            elif filter(token) in node_idx:
                token_idx = str( node_idx[filter(token)] )
                code += 'shared_state[offset+'+token_idx+']'
            
            # Nodes that are in RHS but were not present in the LHS of any line
            elif token.strip() not in operators:
                print('%s was not initialized with an equation. Assigning a random state' % token)
                node_idx[filter(token)] = i
                token_idx = str(i)
                i += 1
                
                code += ' true ' if random.randint(0,1) else ' false '

            # Operators
            else:
                code += operators[token.strip()]

        code += ';\n'



# Printing the header files

initializer = '''
#ifndef INITIALIZER_H
#define INITIALIZER_H
#define N ''' + str(N) + '\n' # i is greater tha N in some cases, due to uninitialized nodes
for node in fixed_node:
    index = str( node_idx[filter(node)] )
    initializer += '\tshared_state[offset+'+index+'] = '+fixed_node[filter(node)].lower()+';\n'
initializer += '#endif'

equations = '''
#ifndef EQUATIONS_H
#define EQUATIONS_H
''' + code + '#endif'

registers = '''
#ifndef REGISTERS_H
#define REGISTERS_H
'''
for node in original_nodes:
    registers += '\tunsigned int ' + node + ' = 0;\n'
registers += '#endif\n'

registers_sum = '''
#ifndef REGISTERS_SUM_H
#define REGISTERS_SUM_H
'''
for node in original_nodes:
    idx = str(node_idx[node])
    registers_sum += '\t\t' + node + ' += shared_state[offset+'+idx+'];\n'
registers_sum += '#endif\n'

registers_reduction = '''
#ifndef REGISTERS_REDUCTION_H
#define REGISTERS_REDUCTION_H
'''
for node in original_nodes:
    idx = str(node_idx[node])
    registers_reduction += '\tatomicAdd(&histogram['+idx+'],'+node+');\n'
registers_reduction += '#endif\n'




with open('initializer.h', 'w') as output:
    output.write(initializer)
with open('equations.h', 'w') as output:
    output.write(equations)
with open('registers.h', 'w') as output:
    output.write(registers)
with open('registers_reduction.h', 'w') as output:
    output.write(registers_reduction)
with open('registers_sum.h', 'w') as output:
    output.write(registers_sum)


Writing equation_to_code.py


## GPU
Em ambas as estratégias, a carga das iterações é dividida entre as *threads* da GPU. Além disso, cada uma delas possui uma cópia inicializada aleatoriamente da rede a ser analisada.

### Naive State Array

Implementação que serve de Baseline para as técnicas de otimização em GPU.

- Código do **kernel** é gerado para um grafo específico;

- Utilização de *array* para representação dos estados e armazenamento do histograma;


In [ ]:
%%writefile GPU_naive.cu

#include <string.h>
#include <stdbool.h>
#include <stdio.h>

#include "initializer.h"
#include "profiler.h"
#include "curand.h"

__global__
void simulate(int iterations, bool *shared_state, bool *temp_state, int *histogram, size_t size) {

    int tId = threadIdx.x + blockDim.x*blockIdx.x;
    if(tId >= size-N) return;

    // Displaces each thread to its respective portion of the array
    int offset = tId*N;
    
    for(int i=0; i<iterations; i++) {

        #include "equations.h"
        
        // Updating network for the next iteration
        #pragma unroll
        for(int j=0; j<N; j++) 
            shared_state[offset+j] = temp_state[offset+j];
        
        // Updating histogram
        #pragma unroll
        for(int j=0; j<N; j++)
            histogram[offset+j] += shared_state[offset+j];

    }

}

__global__
void generate_starting_states(bool *states, float *rand, size_t size) {

    int tId = threadIdx.x + blockIdx.x*blockDim.x;

    // size-N accounts for the last tId iterating N times until the last position
    if(tId >= size-N) return;

    int offset = tId*N;
    #pragma unroll
    for(int i=0; i<N; i++)
        states[offset+i] = rand[offset+i]>=0.5?true:false;

}

int main() {
	
    
    // Kernel variables
    int threads = 128; 
    int blocks = (N+threads-1)/threads;
    printf("threads:%d\tblocks:%d\n", threads, blocks);

    size_t n_bytes = sizeof(bool) * N * threads;
    size_t hist_bytes = sizeof(int) * N * threads;
    size_t size = N*threads;
    
    // Randomizing starting positions
    float *d_rand;
    cudaMalloc((void **) &d_rand, size*sizeof(float));
    
    curandGenerator_t gen;
    curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);
    curandGenerateUniform(gen, d_rand, N*threads);
    
    // Device network and histogram variables
    bool *d_state, *d_next_state;
    int *d_histogram;
    cudaMalloc((void**) &d_state, n_bytes);
    cudaMalloc((void**) &d_next_state, n_bytes);
    cudaMalloc((void**) &d_histogram, hist_bytes);

    generate_starting_states<<<blocks,threads>>>(d_state, d_rand, size);

    float elapsed_time[MAX_POWER10];
    unsigned long long iterations = 10;
    for(int t=0; t<MAX_POWER10; t++) {
        
        int split_workload = iterations / (blocks*threads) + 1;

        //cudaMemset(d_state, 0, n_bytes);
        cudaMemset(d_histogram, 0, hist_bytes);

        GPU_Stopwatch s;
        simulate <<< blocks, threads >>> (split_workload, d_state, d_next_state, d_histogram, size);
        elapsed_time[t] = s.stop();
        
        printf("it:%llu\t\tsplit_wload:%d\t\ttime:%f\n", iterations, split_workload, elapsed_time[t]);
        iterations *= 10;
    }

    // Host variables
    int *thread_histograms, *h_histogram;
    thread_histograms = (int*) malloc(hist_bytes);
    h_histogram = (int*) malloc(N*sizeof(int));
    
    memset(h_histogram, 0, N*sizeof(int));
    cudaMemcpy(thread_histograms, d_histogram, hist_bytes, cudaMemcpyDeviceToHost);
    reduce_histogram(h_histogram, thread_histograms, size, N);
    
    write_csv("GPU_naive.csv", elapsed_time, MAX_POWER10);
    print_histogram(h_histogram, N);

    return 0;
}

Writing GPU_naive.cu


### GPU Sate Array  and Shared Memory

Implementação semelhante ao *baseline*, com a adição de:

- Memória compartilhada para manipulação dos estados 
- Utilização de registradores para armazenamento e cálculo das frequências do histograma.
- Diretiva `atomicAdd();` para redução do histograma
- Utilização reduzida da *global memory*

In [ ]:
%%writefile GPU_shared_reg.cu

#include <string.h>
#include <stdbool.h>
#include <stdio.h>

#include "initializer.h"
#include "profiler.h"
#include "curand.h"

#define THREADS 128
// Talvez seja válido variar com o numero de iterações
#define BLOCKS (N+THREADS-1)/THREADS
#define SHARED_SIZE THREADS*BLOCKS*N

__global__
void simulate(int iterations, bool *state, int *histogram, size_t size) {

    int tId = threadIdx.x + blockDim.x*blockIdx.x;
    if(tId >= size-N) return;

    __shared__ bool shared_state[SHARED_SIZE];
    __shared__ bool temp_state[SHARED_SIZE];

    // Displaces each thread to its respective portion of the array
    int offset = tId*N;

    #include "registers.h"

    // Copying initial state
    for(int i=0; i<N; i++) 
        shared_state[offset+i] = state[offset+i];
    
    for(int i=0; i<iterations; i++) {

        #include "equations.h"

        // Updating network for the next iteration
        #pragma unroll
        for(int j=0; j<N; j++) 
            shared_state[offset+j] = temp_state[offset+j];
        
        #include "registers_sum.h"
        
    }

    #include "registers_reduction.h"

}


__global__
void generate_starting_states(bool *states, float *rand, size_t size) {

    int tId = threadIdx.x + blockIdx.x*blockDim.x;

    // size-N accounts for the last tId iterating N times until the last position
    if(tId >= size-N) return;

    int offset = tId*N;
    #pragma unroll
    for(int i=0; i<N; i++)
        states[offset+i] = rand[offset+i]>=0.5?true:false;

}

int main() {
	
    
    // Kernel variables
    int threads = THREADS; 
    int blocks = BLOCKS;
    printf("N:%d\tthreads:%d\tblocks:%d\n", N, threads, blocks);
    printf("Estimated shared memory needed: %.2fKB\tNvidia max: 64KB\n\n", (threads*blocks*N)/1E3);

    // Default shared is 48KB, this forces more allocation for shared and less for L1
    cudaFuncSetCacheConfig(simulate, cudaFuncCachePreferShared);

    size_t size = N*threads*blocks;
    size_t n_bytes = sizeof(bool) * size;
    size_t hist_bytes = sizeof(int) * N;
    
    // Randomizing starting positions
    float *d_rand;
    cudaMalloc((void **) &d_rand, size*sizeof(float));
    
    curandGenerator_t gen;
    curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);
    curandGenerateUniform(gen, d_rand, size);
    
    // Device network and histogram variables
    bool *d_state;
    int *d_histogram;
    cudaMalloc((void**) &d_state, n_bytes);
    cudaMalloc((void**) &d_histogram, hist_bytes);

    generate_starting_states<<<blocks,threads>>>(d_state, d_rand, size);

    float elapsed_time[MAX_POWER10];
    unsigned long long iterations = 10;
    for(int t=0; t<MAX_POWER10; t++) {
        
        int split_workload = iterations / (blocks*threads) + 1;

        cudaMemset(d_histogram, 0, hist_bytes);

        GPU_Stopwatch s;
        simulate <<< blocks, threads >>> (split_workload, d_state, d_histogram, size);
        elapsed_time[t] = s.stop();
        
        printf("it:%llu\t\tsplit_wload:%d\t\ttime:%f\n", iterations, split_workload, elapsed_time[t]);
        iterations *= 10;
    }
    
    // Host variables
    int *h_histogram;
    h_histogram = (int*) malloc(hist_bytes);
    cudaMemcpy(h_histogram, d_histogram, hist_bytes, cudaMemcpyDeviceToHost);
    
    write_csv("GPU_shared_reg.csv",  elapsed_time, MAX_POWER10);
    print_histogram(h_histogram, N);

    return 0;
}

Writing GPU_shared_reg.cu


 # Analysing

In [ ]:
%%bash
for file in /content/research/boolean-networks/networks/40-100/*/*.txt
do
    echo $file
    python3 equation_to_code.py "$file"
    nvcc GPU_shared_reg.cu -lcurand -o GPU_shared_reg && ./GPU_shared_reg
    nvcc GPU_naive.cu -lcurand -o GPU_naive && ./GPU_naive
done

In [ ]:
!nvcc -arch=sm_75 -Xptxas -v -lcurand GPU_shared_reg.cu

In [ ]:
# Fetching data from GPU and CPU benchmarks
cpu = []
gpu_naive= []
gpu_shared_reg= []

strategy = {'CPU.csv':[], 'GPU_naive.csv':[], 'GPU_shared_reg.csv':[]}

for key in strategy:
    try:
        with open(key, "r") as file:
            # i controls how many measurements to skip
            i = 0
            for value in file.readline().split(","):
                if i<4:
                    i += 1
                    continue  
                strategy[key].append(float(value)/1000)
    except OSError:
        print('Could not open/read strategy ' + key + '. Skipping it...')
        continue

## Results 
No gráfico abaixo, podemos notar que a estratégia que utilizou array de estados pré-compilado em conjunto com memória compartilhada foi a que obteve melhor desempenho, seguida dos ponteiros de função, da implementação *naive* em GPU e da implementação da simulação em CPU.

In [ ]:
import matplotlib.pyplot as plt

# Number of elements utilized
labels = ["1E5", "1E6", "1E7", "1E8", "1E9"]
param = {'CPU.csv':('go-','CPU'), 'GPU_naive.csv':('bo-','GPU Naive'), 'GPU_shared_reg.csv':('ro-','GPU Shared+Registers'), 3:('mo-','Unknown')}

# Graph plotting settings
plt.figure(figsize=(16,8))
for key in strategy:
    try:
        plt.plot(labels, strategy[key], param[key][0], linewidth=2, label=param[key][1])
    except (KeyError, ValueError):
        print('Skipping ' + key)
        continue

plt.xlabel("# de iterações em potência de 10")
plt.ylabel("Runtime em Segundos")
plt.title("CPU vs. GPU Gene Simulation Runtime")
plt.legend(loc="upper left")

Abaixo segue a tabela de *speedup* aproximado do código comparado à CPU (A FAZER)

GPU Strategy | Speedup
-------------|---------
Naive        |     2x
Global Memory| 12x
Shared+Registers  |     45x


Observando a tabela acima, mesmo ao considerar a estratégia mais simples, fica clara a contribuição da utilização de GPUs para a Simulação de Redes Booleanas. Ainda que o resultado seja satisfatório, outras otimizações como a representação da rede em bits, *profiling* usando *nsight* e refatoração de código podem ser realizadas na presente implementação com intuito de alcançar ainda mais performance. Por fim, para trabalhos futuros é necessário tornar a geração de código pré-compilado simples e automática, de modo que esta ferramenta possa ser utilizada por pesquisadores da área sem necessidade de conhecimentos específicos.